In [93]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import mxnet as mx
from mxnet import gluon, np, npx, nd, autograd
from mxnet.gluon import nn
from mxnet import init
import statsmodels.api as sm
from statsmodels.tools import eval_measures
import random
import numpy
from sklearn.metrics import mean_squared_error
npx.set_np()

# Load data

In [2]:
#Load data
with open('msd_full.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test']

# reserve the last 10% of the training dataset as the validation set, and the remaining 90% as the subtraining set.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [3]:
# Y de mean
Y_train_mean = Y_train.mean()
de_mean_Y_train = Y_train - Y_train_mean
de_mean_Y_test = Y_test - Y_train_mean

In [4]:
# first 1000
X_train_10000 = X_train[:10000]
Y_train_10000 = Y_train[:10000]
de_mean_Y_train_10000 = de_mean_Y_train[:10000]

In [5]:
rmse_dict = {}

# Q1
Train and tune the models listed above. Report test RMSE for each model setting.

In [6]:
def relu(X):
    return np.maximum(X, 0)

In [7]:
def SGD(params, lr, batch_size):
    for param in params:
        param[:] = param - lr * param.grad / batch_size

In [8]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [60]:
def l2_penalty(w_list):
    total = 0
    for w in w_list:
        total = total + (w**2).sum() / 2
    return total / len(w_list)

## Case OLS

In [9]:
model = sm.OLS(Y_train_10000,X_train_10000).fit()
y_pred = model.predict(X_val)
ols_rmse = eval_measures.rmse(Y_val, y_pred)
rmse_dict['OLS'] = ols_rmse
print(ols_rmse)

2009.9896859926644


---

In [10]:
# first 1000
X_train_10000 = np.array(X_train_10000)
Y_train_10000 = np.array(Y_train_10000)
de_mean_Y_train_10000 = np.array(de_mean_Y_train_10000)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

In [11]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = np.array(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

## Case MLP_0_dm

In [12]:
def linreg(X, w, b):
    return np.dot(X, w) + b

In [13]:
w = np.random.normal(0, 0.01, (90, 1)).astype('float64')
b = np.zeros(1)
w.attach_grad()
b.attach_grad()

In [14]:
lr = 0.03  # Learning rate
num_epochs = 4  # Number of iterations
batch_size = 10


for epoch in range(num_epochs):
    # Assuming the number of examples can be divided by the batch size, all
    # the examples in the training dataset are used once in one epoch
    # iteration. The features and tags of minibatch examples are given by X
    # and y respectively
    for X, y in data_iter(batch_size, X_train_10000, de_mean_Y_train_10000):
        with autograd.record():
            l = squared_loss(linreg(X, w, b), y)  # Minibatch loss in X and y
        l.backward()  # Compute gradient on l with respect to [w, b]
        SGD([w, b], lr, batch_size)  # Update parameters using their gradient


In [18]:
y_pred = linreg(X_val, w, b)
loss = squared_loss(linreg(X_val, w, b), Y_val)

mlp_0_dm_rmse = mx.metric.RMSE()
mlp_0_dm_rmse.update(labels = Y_val, preds = y_pred)
print(mlp_0_dm_rmse)
rmse_dict['mlp_0_dm'] = mlp_0_dm_rmse

EvalMetric: {'rmse': 1998.4381879230896}


## Case MLP_1_dm	

In [47]:
num_inputs, num_outputs, num_hiddens = X_train_10000.shape[1], 1, 45

W1 = np.random.normal(scale=0.01, size=(num_inputs, num_hiddens)).astype('float64')
b1 = np.zeros(num_hiddens)
W2 = np.random.normal(scale=0.01, size=(num_hiddens, num_outputs)).astype('float64')
b2 = np.zeros(num_outputs)
params = [W1, b1, W2, b2]

for param in params:
    param.attach_grad()

In [48]:
def mlp_1_net(X):
    X = X.reshape(-1, num_inputs)
    H = relu(np.dot(X, W1) + b1)
    return np.dot(H, W2) + b2

In [49]:
lr = 0.03  # Learning rate
num_epochs = 4  # Number of iterations
batch_size = 10

for epoch in range(num_epochs):
    # Assuming the number of examples can be divided by the batch size, all
    # the examples in the training dataset are used once in one epoch
    # iteration. The features and tags of minibatch examples are given by X
    # and y respectively
    for X, y in data_iter(batch_size, X_train_10000, de_mean_Y_train_10000):
        with autograd.record():
            l = squared_loss(mlp_1_net(X), y)  # Minibatch loss in X and y
        l.backward()  # Compute gradient on l with respect to [w, b]
        SGD([W1, b1, W2, b2], lr, batch_size)  # Update parameters using their gradient

In [52]:
y_pred = mlp_1_net(X_val)
loss = squared_loss(mlp_1_net(X_val), Y_val)

mlp_1_dm_rmse = mx.metric.RMSE()
mlp_1_dm_rmse.update(labels = Y_val, preds = y_pred)
print(mlp_1_dm_rmse)
rmse_dict['mlp_1_dm'] = mlp_1_dm_rmse

EvalMetric: {'rmse': 1998.4392694005303}


## Case MLP_2_dm

In [54]:
num_inputs, num_outputs, num_hiddens = X_train_10000.shape[1], 1, 45

W1 = np.random.normal(scale=0.01, size=(num_inputs, num_hiddens)).astype('float64')
b1 = np.zeros(num_hiddens)
W2 = np.random.normal(scale=0.01, size=(num_hiddens, num_hiddens)).astype('float64')
b2 = np.zeros(num_outputs)
W3 = np.random.normal(scale=0.01, size=(num_hiddens, num_outputs)).astype('float64')
b3 = np.zeros(num_outputs)
params = [W1, b1, W2, b2, W3, b3]

for param in params:
    param.attach_grad()

In [55]:
def mlp_2_net(X):
    X = X.reshape(-1, num_inputs)
    H1 = relu(np.dot(X, W1) + b1)
    H2 = relu(np.dot(H1, W2) + b2)
    return np.dot(H2, W3) + b3

In [56]:
lr = 0.03  # Learning rate
num_epochs = 4  # Number of iterations
batch_size = 10

for epoch in range(num_epochs):
    # Assuming the number of examples can be divided by the batch size, all
    # the examples in the training dataset are used once in one epoch
    # iteration. The features and tags of minibatch examples are given by X
    # and y respectively
    for X, y in data_iter(batch_size, X_train_10000, de_mean_Y_train_10000):
        with autograd.record():
            l = squared_loss(mlp_2_net(X), y)  # Minibatch loss in X and y
        l.backward()  # Compute gradient on l with respect to [w, b]
        SGD([W1, b1, W2, b2, W3, b3], lr, batch_size)  # Update parameters using their gradient

In [59]:
y_pred = mlp_2_net(X_val)
loss = squared_loss(linreg(X_val, w, b), Y_val)

mlp_2_dm_rmse = mx.metric.RMSE()
mlp_2_dm_rmse.update(labels = Y_val, preds = y_pred)
print(mlp_2_dm_rmse)
rmse_dict['mlp_2_dm'] = mlp_2_dm_rmse

EvalMetric: {'rmse': 1998.4379025121495}


## Case MLP_2_dm_L2

In [69]:
def init_params():
    W1 = np.random.normal(scale=0.01, size=(num_inputs, num_hiddens)).astype('float64')
    b1 = np.zeros(num_hiddens)
    W2 = np.random.normal(scale=0.01, size=(num_hiddens, num_hiddens)).astype('float64')
    b2 = np.zeros(num_outputs)
    W3 = np.random.normal(scale=0.01, size=(num_hiddens, num_outputs)).astype('float64')
    b3 = np.zeros(num_outputs)
    params = [W1, b1, W2, b2, W3, b3]
    
    for param in params:
        param.attach_grad()
    return params

In [104]:
def train_l2(lambd):
    W1, b1, W2, b2, W3, b3 = init_params()
    net, loss = lambda X: mlp_2_net(X), squared_loss
    num_epochs, lr = 4, 0.003
    #animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
    #                        xlim=[1, num_epochs], legend=['train', 'test'])
    for epoch in range(1, num_epochs + 1):
        for X, y in data_iter(batch_size, X_train_10000, de_mean_Y_train_10000):
            with autograd.record():
                # The L2 norm penalty term has been added, and broadcasting
                # makes l2_penalty(w) a vector whose length is batch_size
                l = loss(net(X), y) + lambd * l2_penalty([W1, W2, W3])
            l.backward()
            SGD([W1, b1, W2, b2, W3, b3], lr, batch_size)
        #if epoch % 5 == 0:
        #    animator.add(epoch, (d2l.evaluate_loss(net, train_iter, loss),
        #                         d2l.evaluate_loss(net, test_iter, loss)))
    y_pred = mlp_2_net(X_val)
    loss = squared_loss(mlp_2_net(X_val)
    mlp_2_dm_l2_rmse = mx.metric.RMSE()
    mlp_2_dm_l2_rmse.update(labels = Y_val, preds = y_pred)
    print(mlp_2_dm_rmse)

SyntaxError: invalid syntax (<ipython-input-104-80a875f0cfbc>, line 20)

In [102]:
X_train_10000 = np.array(X_train_10000, dtype='float32')
Y_train_10000 = np.array(Y_train_10000, dtype='float32')
de_mean_Y_train_10000 = np.array(de_mean_Y_train_10000, dtype='float32')
X_val = np.array(X_val, dtype='float32')
Y_val = np.array(Y_val, dtype='float32')

In [100]:
def train_gluon(wd):
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize(init.Normal(sigma=1))
    loss = gluon.loss.L2Loss()
    num_epochs, lr = 4, 0.003
    trainer = gluon.Trainer(net.collect_params(), 'sgd',
                            {'learning_rate': lr, 'wd': wd})
    # The bias parameter has not decayed. Bias names generally end with "bias"
    net.collect_params('.*bias').setattr('wd_mult', 0)

    #animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
    #                        xlim=[1, num_epochs], legend=['train', 'test'])
    for epoch in range(1, num_epochs+1):
        for X, y in data_iter(batch_size, X_train_10000, de_mean_Y_train_10000):
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        #if epoch % 5 == 0:
        #    animator.add(epoch, (d2l.evaluate_loss(net, train_iter, loss),
        #                         d2l.evaluate_loss(net, test_iter, loss)))
    print('L1 norm of w:', np.abs(net[0].weight.data()).sum())

In [101]:
train_gluon(0)

L1 norm of w: 

MXNetError: [13:48:22] include/mxnet/./tensor_blob.h:255: Check failed: mshadow::DataType<DType>::kFlag == type_flag_: TBlob.get_with_shape: data type do not match specified type.Expected: 0 v.s. given 1
Stack trace:
  [bt] (0) 1   libmxnet.so                         0x0000001a171c0bd9 libmxnet.so + 31705
  [bt] (1) 2   libmxnet.so                         0x0000001a171d0f1c mxnet::op::NDArrayOpProp::~NDArrayOpProp() + 44396
  [bt] (2) 3   libmxnet.so                         0x0000001a1837406a void mxnet::op::BinaryBroadcastCsrDnsDnsImpl<mshadow::cpu, mxnet::op::mshadow_op::plus>(mxnet::OpContext const&, mxnet::NDArray const&, mxnet::NDArray const&, mxnet::OpReqType, mxnet::NDArray const&, mxnet::TShape const&, mxnet::TShape const&, mxnet::TShape const&, int, bool) + 33274
  [bt] (3) 4   libmxnet.so                         0x0000001a1835dfd9 void mxnet::op::BinaryBroadcastComputeDenseEx<mshadow::cpu, mxnet::op::mshadow_op::plus>(nnvm::NodeAttrs const&, mxnet::OpContext const&, std::__1: